# **Chatbot - NLP and Deep Learning**

For this project, we will use the PyTorch library.

---

## 1. Theory and NLP concepts

We will talk about stemming, tokenization, bag of words.

First, we put all words (of each patterns) into an array.

- **Bag of words :** For each different pattern, we create an array w/ the same size as the all words array. If this word is included into the all words array, we put a 1 at his position, 0 otherwise.
- **Tokenization :** Splitting string into meaningful units (e.g. words, punctuation characters, numbers)
- **Stemming :** Generate the root form of the words. It is an heuristic that chops of the ends off of words. 

### **Whole NLP pre-processing pipeline :** 

At the beginning, we have the Whole sentence, then we tokenize it. We lower all the words, then we stem the words. We then exclude punctuation characters. And based on this array, we calculate the bag of words. 

---

## 2. Create training data

---

## 3. PyTorch model and training

---

## 4. Save and load model and implement the chat